<center><h1>Shah_Unnati_HW3</h1></center>
<br>
<br>

Name: Unnati Bhaven Shah
<br>
Github Username: unnatibshah
<br>
USC ID: 4512915005

## 1. Time Series Classification Part 1: Feature Creation/Extraction

### (a) Download Data

Package imports

In [20]:
import os
import re
import numpy as np
import pandas as pd
from sklearn.utils import resample

Get the AReM Data Set

In [21]:
data_root = "../data/AReM/"

actvt_dirs = ["bending1", "bending2", "cycling", "lying", "sitting", "standing", "walking"]
orig_columns = ['# Columns: time', 'avg_rss12', 'var_rss12', 'avg_rss13', 'var_rss13', 'avg_rss23', 'var_rss23']


### (b) Test and Train Data

In [22]:
# function to traverse data folder and split file paths in to train group and test group
def split_train_test(root):
    train_files = []
    test_files = []
    for root, _, file in os.walk(data_root):
        for f in file:
            if (os.path.splitext(f)[1] == ".csv"):
                csv_idx = int(re.findall("\d+", f)[0])
                file_path = os.path.join(root, f)
                activity = os.path.split(root)[-1]
                if (csv_idx <= 2 or (activity not in ["bending1", "bending2"] and csv_idx == 3)):
                    test_files.append(file_path)
                else:
                    train_files.append(file_path)
    return train_files, test_files

In [23]:
# split instances
train_files, test_files = split_train_test(data_root)

In [24]:
train_files

['../data/AReM/bending1/dataset7.csv',
 '../data/AReM/bending1/dataset6.csv',
 '../data/AReM/bending1/dataset4.csv',
 '../data/AReM/bending1/dataset5.csv',
 '../data/AReM/bending1/dataset3.csv',
 '../data/AReM/walking/dataset7.csv',
 '../data/AReM/walking/dataset6.csv',
 '../data/AReM/walking/dataset4.csv',
 '../data/AReM/walking/dataset5.csv',
 '../data/AReM/walking/dataset10.csv',
 '../data/AReM/walking/dataset11.csv',
 '../data/AReM/walking/dataset13.csv',
 '../data/AReM/walking/dataset12.csv',
 '../data/AReM/walking/dataset15.csv',
 '../data/AReM/walking/dataset14.csv',
 '../data/AReM/walking/dataset8.csv',
 '../data/AReM/walking/dataset9.csv',
 '../data/AReM/bending2/dataset6.csv',
 '../data/AReM/bending2/dataset4.csv',
 '../data/AReM/bending2/dataset5.csv',
 '../data/AReM/bending2/dataset3.csv',
 '../data/AReM/standing/dataset7.csv',
 '../data/AReM/standing/dataset6.csv',
 '../data/AReM/standing/dataset4.csv',
 '../data/AReM/standing/dataset5.csv',
 '../data/AReM/standing/dataset

In [25]:
test_files

['../data/AReM/bending1/dataset1.csv',
 '../data/AReM/bending1/dataset2.csv',
 '../data/AReM/walking/dataset1.csv',
 '../data/AReM/walking/dataset2.csv',
 '../data/AReM/walking/dataset3.csv',
 '../data/AReM/bending2/dataset1.csv',
 '../data/AReM/bending2/dataset2.csv',
 '../data/AReM/standing/dataset1.csv',
 '../data/AReM/standing/dataset2.csv',
 '../data/AReM/standing/dataset3.csv',
 '../data/AReM/sitting/dataset1.csv',
 '../data/AReM/sitting/dataset2.csv',
 '../data/AReM/sitting/dataset3.csv',
 '../data/AReM/lying/dataset1.csv',
 '../data/AReM/lying/dataset2.csv',
 '../data/AReM/lying/dataset3.csv',
 '../data/AReM/cycling/dataset1.csv',
 '../data/AReM/cycling/dataset2.csv',
 '../data/AReM/cycling/dataset3.csv']

### (c) Feature Extraction

#### i. Research

    maximum
    minimum
    mean
    median
    standard deviation
    first quartile
    third quartile
    correlation structure
    distribution
    entropy
    stationarity
    scaling properties

#### ii. Extraction

In [26]:
def load_data(files, feature, orig_columns, segs=1, standard=False):
    import csv
    sniffer = csv.Sniffer()
    
    # generate columns of describes and label with indecies
    describe_order = ['mean', 'std', 'min', '_1stquart_', 'median', '_3rdquart_','max']
    desc_order_idx = [desc + str(i) for i in range(1, 6 * segs + 1) for desc in describe_order]
    
    instances  = []
    labels = []
    for csv in files:
        # get the label of the instance read from current csv
        parent_folder = os.path.dirname(csv)
        activity = os.path.split(parent_folder)[-1]
        labels.append(activity)
        
        # identidy the separater of the csv file
        sep = sniffer.sniff(open(csv).read()).delimiter
        
        # error_bad_lines=False used to 
        if sep == ",":
            cur_instance = pd.read_csv(csv, skiprows=5, header=None, 
                                      on_bad_lines='warn')
        else:
            cur_instance = pd.read_csv(csv, skiprows=5, header=None, sep="\s+", 
                                      on_bad_lines='warn')
        
        # restore the column of csv file as we skip 5th row which is headers in the csv
        # however, they will becomes mass when separater is space. Thus, skip it
        cur_instance.columns = orig_columns
        
        # break 6 time series into equal length, becomes 6*segs series
        seg_length = int(cur_instance.shape[0] / segs)
        segs_describe = []
        for i in range(segs):
            cur_seg = cur_instance[i * seg_length : min((i + 1) * seg_length, cur_instance.shape[0])]
            cur_describe = cur_seg.describe().drop('count').drop(columns="# Columns: time").T
            segs_describe.append(cur_describe.values.flatten())
        # complete current instance reading
        instances.append(np.concatenate(segs_describe, axis=None))
    
    # convert to dataframe and reorder the columns based on given feature list
    feature_df = pd.DataFrame(instances, columns=desc_order_idx)
    feature_df = feature_df.loc[:, feature]
    
    if standard:
        feature_df = feature_df.apply(lambda col: (col-col.mean()) / (np.std(col) + 1e-9), axis = 0)
    
    # add label to complete the dataset
    feature_df['label'] = pd.Series(labels)
    
    return feature_df

In [27]:
# reorder the columns
statistics = ['min','max','mean','median','std','_1stquart_', '_3rdquart_']
feature = [stat + str(i) for i in range(1, 7) for stat in statistics]

# load dataset as required
train_data = load_data(train_files, feature, orig_columns)
test_data = load_data(test_files, feature, orig_columns)


Skipping line 485: expected 7 fields, saw 8

Skipping line 485: expected 7 fields, saw 8



In [28]:
train_data

,min1,max1,mean1,median1,std1,_1stquart_1,_3rdquart_1,min2,max2,mean2,...,_1stquart_5,_3rdquart_5,min6,max6,mean6,median6,std6,_1stquart_6,_3rdquart_6,label
0,36.25,48.00,43.969125,44.50,1.618364,43.310,44.67,0.0,1.50,0.413125,...,20.5000,23.7500,0.0,2.96,0.555312,0.490,0.487826,0.0000,0.830,bending1
1,37.00,48.00,43.454958,43.25,1.386098,42.500,45.00,0.0,1.58,0.378083,...,22.2500,24.0000,0.0,5.26,0.679646,0.500,0.622534,0.4300,0.870,bending1
2,33.00,47.75,42.179812,43.50,3.670666,39.150,45.00,0.0,3.00,0.696042,...,30.4575,36.3300,0.0,2.18,0.613521,0.500,0.524317,0.0000,1.000,bending1
3,33.00,45.75,41.678063,41.75,2.243490,41.330,42.75,0.0,2.83,0.535979,...,28.4575,31.2500,0.0,1.79,0.383292,0.430,0.389164,0.0000,0.500,bending1
4,35.00,47.40,43.954500,44.33,1.558835,43.000,45.00,0.0,1.70,0.426250,...,35.3625,36.5000,0.0,1.79,0.493292,0.430,0.513506,0.0000,0.940,bending1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,18.50,44.25,35.752354,36.00,4.614802,33.000,39.33,0.0,12.60,3.328104,...,14.0000,18.0625,0.0,9.39,3.069667,2.770,1.748326,1.7975,4.060,cycling
65,24.25,45.00,37.177042,36.25,3.581301,34.500,40.25,0.0,8.58,2.374208,...,17.9500,21.7500,0.0,9.34,2.921729,2.500,1.852600,1.5000,3.900,cycling
66,23.33,43.50,36.248768,36.75,3.824632,33.415,39.25,0.0,9.71,2.737307,...,15.7500,21.0000,0.0,11.15,3.532463,3.110,1.965267,2.1700,4.625,cycling
67,26.25,44.25,36.957458,36.29,3.434863,34.500,40.25,0.0,8.64,2.420083,...,14.0000,18.2500,0.0,8.34,2.934625,2.525,1.631380,1.6600,4.030,cycling


In [29]:
test_data

,min1,max1,mean1,median1,std1,_1stquart_1,_3rdquart_1,min2,max2,mean2,...,_1stquart_5,_3rdquart_5,min6,max6,mean6,median6,std6,_1stquart_6,_3rdquart_6,label
0,37.25,45.00,40.624792,40.500,1.476967,39.2500,42.0000,0.0,1.30,0.358604,...,33.0000,36.0000,0.0,1.92,0.570583,0.430,0.582915,0.0000,1.300,bending1
1,38.00,45.67,42.812812,42.500,1.435550,42.0000,43.6700,0.0,1.22,0.372437,...,32.0000,34.5000,0.0,3.11,0.571083,0.430,0.601010,0.0000,1.300,bending1
2,19.33,43.50,34.227771,35.500,4.889576,30.5000,37.7500,0.0,14.50,3.995729,...,14.7500,18.6700,0.0,9.74,3.394125,3.100,1.792090,2.1050,4.425,walking
3,12.50,45.00,33.509729,34.125,4.850923,30.5000,36.7500,0.0,13.05,4.450771,...,14.6275,18.7500,0.0,8.96,3.378479,3.085,1.787360,2.0600,4.440,walking
4,15.00,46.75,34.660583,35.000,5.315110,31.0000,38.2500,0.0,13.44,4.200896,...,14.2500,18.5000,0.0,8.99,3.244396,3.000,1.630983,2.1200,4.240,walking
5,12.75,51.00,24.562958,24.250,3.737514,23.1875,26.5000,0.0,6.87,0.590833,...,20.5000,27.0000,0.0,4.97,0.700188,0.500,0.693720,0.4300,0.870,bending2
6,0.00,42.75,27.464604,28.000,3.583582,25.5000,30.0000,0.0,7.76,0.449708,...,15.0000,20.7500,0.0,6.76,1.122125,0.830,1.012342,0.4700,1.300,bending2
7,33.33,48.00,44.334729,45.000,2.476940,42.2500,46.5000,0.0,3.90,0.432958,...,9.3300,17.7500,0.0,5.02,0.933000,0.830,0.673609,0.4700,1.250,standing
8,35.50,46.25,43.174938,43.670,1.989052,42.5000,44.5000,0.0,2.12,0.506583,...,12.7500,16.5000,0.0,5.72,0.911979,0.830,0.666161,0.4700,1.220,standing
9,32.75,47.00,42.760562,44.500,3.398919,41.3300,45.3725,0.0,3.34,0.486167,...,13.0000,18.5650,0.0,5.73,0.842271,0.710,0.722165,0.4300,1.090,standing


#### iii. Standard Deviation

In [30]:
train_data.describe().loc['std']

min1           8.794295
max1           4.429182
mean1          4.917692
median1        4.956111
std1           1.758670
_1stquart_1    5.731647
_3rdquart_1    4.783645
min2           0.000000
max2           5.147841
mean2          1.600701
median2        1.436960
std2           0.902808
_1stquart_2    0.952201
_3rdquart_2    2.158416
min3           3.053869
max3           4.759853
mean3          3.863097
median3        3.845730
std3           0.995959
_1stquart_3    4.145255
_3rdquart_3    3.946023
min4           0.000000
max4           2.302408
mean4          1.179861
median4        1.150092
std4           0.473576
_1stquart_4    0.842501
_3rdquart_4    1.566564
min5           5.368786
max5           5.449726
mean5          5.120426
median5        5.267414
std5           1.057998
_1stquart_5    5.543882
_3rdquart_5    4.957231
min6           0.051766
max6           2.540166
mean6          1.171401
median6        1.104626
std6           0.519420
_1stquart_6    0.774358
_3rdquart_6    1

In [31]:
test_data.describe().loc['std']

min1           12.136206
max1            4.379342
mean1           6.790086
median1         7.088085
std1            1.869285
_1stquart_1     7.673052
_3rdquart_1     6.385720
min2            0.000000
max2            4.870395
mean2           1.500529
median2         1.345903
std2            0.832472
_1stquart_2     0.941307
_3rdquart_2     2.028482
min3            2.644618
max3            5.393220
mean3           4.588252
median3         4.753522
std3            0.760779
_1stquart_3     4.554121
_3rdquart_3     5.014339
min4            0.000000
max4            1.733937
mean4           1.146338
median4         1.161705
std4            0.409008
_1stquart_4     0.867088
_3rdquart_4     1.541943
min5            8.252947
max5            6.782153
mean5           7.366781
median5         7.438749
std5            0.919274
_1stquart_5     7.823870
_3rdquart_5     7.259433
min6            0.000000
max6            2.505306
mean6           1.119410
median6         1.037811
std6            0.525263


In [32]:
data = train_data.iloc[:, :-1]
ans = []
for sample in range(1000):
    resampled = resample(data)
    temp = resampled.apply(lambda col: np.std(col))
    ans.append(temp)

ans = pd.DataFrame(ans)
ans.columns = list(data.columns)
interval = ans.apply(lambda col : (np.percentile(col, 5), np.percentile(col, 95)), axis=0)
interval.apply(np.around, args=(2,))
interval = interval.T
interval.columns = ['low', 'high']
interval

,low,high
min1,7.462672,9.972123
max1,3.303282,5.364175
mean1,4.287674,5.371990
median1,4.300557,5.421138
std1,1.535862,1.935546
_1stquart_1,5.120450,6.143576
_3rdquart_1,3.919821,5.497547
min2,0.000000,0.000000
max2,4.586054,5.478185
mean2,1.372921,1.719476


#### iv. Select Features



Since this is a regression problem, the choice of important time-domain features will be majorly driven by the feasibility of demarkation and there should be relatively clear separated distribution and hence the choice would be -

    min
    max
    3rd quartile



## 2. ISLR 3.7.4

### (a) Linear Train

Due to the direct relationship between the number of predictors and model fit, it is expected that the Residual Sum of Squares (RSS) in linear regression would be lower compared to cubic regression. This is because cubic regression incorporates more predictors, which typically results in a superior model fit.

### (b) Linear Test

Increased predictor variables tend to result in overfitting when applied to test data. Consequently, in a test scenario, it is expected that linear regression would yield a lower RSS because it is more likely to produce a relatively accurate regression outcome.

### (c) Not Linear Train

The core principle remains consistent: having more predictors typically leads to a reduction in RSS, and therefore, the RSS for cubic regression would be lower. Cubic regression offers greater flexibility due to its increased number of predictors.

### (d) Not Linear Testing

The adequacy of the information provided depends on the determination of which regression model is closer to the actual answer.

## 3. ISLR 3.7.3 - Extra Practice 

## 3. ISLR 3.7.5 - Extra Practice 